In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sb
%matplotlib inline

In [2]:
import sklearn
from sklearn import cross_validation
from sklearn import linear_model

Repeating initial steps. Going to try applying logistic regression this time. Going to try one train/test split first. If accuracy is more than 50%, I will try multiple splits.

In [3]:
trainset = pd.read_csv("train.csv")
trainframe = pd.DataFrame(trainset)
trainframe["Age"]= trainframe["Age"].fillna(trainframe["Age"].median())

Converting column "Sex" to numerical categories

In [4]:
trainframe.loc[trainframe["Sex"] =="male", "Sex"] = 0
trainframe.loc[trainframe["Sex"] == "female", "Sex"] = 1

Converting column "Embarked" to numerical categories

In [5]:
trainframe.loc[trainframe["Embarked"] == "S", "Embarked"] = 0
trainframe.loc[trainframe["Embarked"] == "C", "Embarked"] = 1
trainframe.loc[trainframe["Embarked"] == "Q", "Embarked"] = 2
trainframe["Embarked"] = trainframe["Embarked"].fillna(trainframe["Embarked"].median())

Data cleaning done. Now to split dataset in to train and test datasets.

In [7]:
X = trainframe[["Pclass", "Age", "SibSp", "Parch", "Embarked"]]
Y = trainframe[["Survived"]]

In [9]:
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(X, Y, test_size = 0.2, random_state = 10)

In [13]:
model = linear_model.LogisticRegression(random_state = 1)

In [14]:
model.fit(X_train, Y_train)

/home/avanti/anaconda/envs/py3k/lib/python3.3/site-packages/sklearn/preprocessing/label.py:125: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=1, tol=0.0001)

In [18]:
Y_train_flat = np.ravel(Y_train).tolist()
model.fit(X_train, Y_train_flat)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=1, tol=0.0001)

following procedure similar to titanic_linear

In [20]:
test_predictions = model.predict(X_test)
predictions = [1 if x >0.5 else 0 for x in  test_predictions]
Y_test_flat = np.ravel(Y_test).tolist()
accuracy = [1 if predictions[i] == Y_test_flat[i] else 0 for i in range(len(predictions                                                                    ))]
print(sum(accuracy)/len(accuracy))

0.7653631284916201


Now adding back a couple of columns we did not have before.

In [21]:
X = trainframe[["Pclass", "Age", "Sex", "SibSp", "Parch", "Fare", "Embarked"]]
Y = trainframe[["Survived"]]
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(X, Y, test_size = 0.2, random_state = 10)
Y_train_flat = np.ravel(Y_train).tolist()
model.fit(X_train, Y_train_flat)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=1, tol=0.0001)

In [22]:
test_predictions = model.predict(X_test)
predictions = [1 if x >0.5 else 0 for x in  test_predictions]
Y_test_flat = np.ravel(Y_test).tolist()
accuracy = [1 if predictions[i] == Y_test_flat[i] else 0 for i in range(len(predictions                                                                    ))]
print(sum(accuracy)/len(accuracy))

0.8212290502793296


For linear regression, we seemed to get a better result with fewer features.

In [24]:
X = trainframe[["Pclass", "Age", "Sex", "Fare", "Embarked"]]
Y = trainframe[["Survived"]]
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(X, Y, test_size = 0.2, random_state = 10)
Y_train_flat = np.ravel(Y_train).tolist()
model.fit(X_train, Y_train_flat)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=1, tol=0.0001)

In [25]:
test_predictions = model.predict(X_test)
predictions = [1 if x >0.5 else 0 for x in  test_predictions]
Y_test_flat = np.ravel(Y_test).tolist()
accuracy = [1 if predictions[i] == Y_test_flat[i] else 0 for i in range(len(predictions                                                                    ))]
print(sum(accuracy)/len(accuracy))

0.8156424581005587


Not a big difference, and it is slightly worse. Now to try the kfold cross validation method.

In [26]:
framesize = trainframe.shape

In [27]:
trainfolds = sklearn.cross_validation.KFold(framesize[0], n_folds = 5, random_state = 1)
predictions = []
predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
model = linear_model.LogisticRegression()

In [28]:
for train, test in trainfolds:
    train_predictors = trainframe[predictors].iloc[train, :]
    train_survival = trainframe["Survived"].iloc[train]
    model.fit(train_predictors, train_survival)
    survival = model.predict(trainframe[predictors].iloc[test, :])
    predictions.append(survival)

In [29]:
len(predictions)

5

In [30]:
len(predictions[0])

179

In [31]:
predictions = np.concatenate(predictions, axis = 0 )

In [32]:
folded_predictions = [1 if x >0.5 else 0 for x in predictions]

In [33]:
accuracy = [1 if folded_predictions[i] == trainframe["Survived"][i] else 0 for i in range(len(predictions                                                                    ))]
print(sum(accuracy)/len(accuracy))

0.7946127946127947


trying a different set of predictors

In [37]:
predictions = []
predictors = ["Pclass", "Sex", "Age", "Fare"]

In [38]:
for train, test in trainfolds:
    train_predictors = trainframe[predictors].iloc[train, :]
    train_survival = trainframe["Survived"].iloc[train]
    model.fit(train_predictors, train_survival)
    survival = model.predict(trainframe[predictors].iloc[test, :])
    predictions.append(survival)

In [39]:
predictions = np.concatenate(predictions, axis = 0 )
folded_predictions = [1 if x >0.5 else 0 for x in predictions]
accuracy = [1 if folded_predictions[i] == trainframe["Survived"][i] else 0 for i in range(len(predictions                                                                    ))]
print(sum(accuracy)/len(accuracy))

0.7912457912457912
